# Notes: changed

This code can perform the following tasks:


*   Tune a CNN to directly reconstruct PET images from Sinograms (find a set of hyperparameters)
*   Train a network with a given set of hyperparameters
*   Test the network and record MSE and SSIM values for each image tested
*   Visualize the data and test results
*   Plot training curves, metric histograms, example images

The code is organized into sections. The important sections that you can edit are:


> **User Parameters** - Edit important user parameters and decide what the code will do

> **Configuration Dicts: Supervisory** - Dictionary for supervised learning. Make sure this matches the CNN loaded by the checkpoint file, if you are loading from a checkpoint.

In addition to these, you may find that running a single cell is useful when all variables/classes/files have been loaded into memory. This can be quicker than running everything from scratch.

The cells nested under **Analysis Functions** each have their own changeable parameters.

*Notes:*

*1) Raytune in particular is constantly changing. Therefore, if you are running this code after the authors have ceased maintaining it and there are errors, these are likely due to RayTune classes, methods, or functions being changed. Unfortunately, these seem to happen on a regular basis, as the code is relatively new.*

*2) This code was originally written to tune/train/test not just sinogram to image supervisory networks (sinogram-->image), but also image to sinogram supervisory networks, GANs, CycleGANs, and Cycle + Supervisory networks. These latter capabilities have not been updated, but much of the code survives for this functionality. In the future, the code may be updated once again have these capabilities.*


GPUs
====
From best to worst:

V100 - 6.92/hr

L4 - 2.15/hr

T4 - 1.7/hr

v6e-1 TPU - 4.21/hr

v5e-1 TPU - 4.11/hr

v2-8 TPU - 1.82/hr


# User Parameters

## General

## Tuning

## Training

## Testing

## Visualizing

# Setup Notebook

In [6]:
import os, sys, glob, importlib, inspect, types, subprocess, pkgutil

def sense_colab():
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False
    return IN_COLAB

def sense_device(device='sense'):
    if device == 'sense':
        if torch.cuda.is_available():
            device = 'cuda'
        else:
            device = 'cpu'
    elif device == 'cpu':
        device = 'cpu'
    elif device == 'cuda':
        device = 'cuda'
    return device

def install_required_packages_ray_version(IN_COLAB, force_reinstall=False, include_optional=True, ray_version=None):
    """
    Installs required Python packages efficiently.
    - Detects if running in Colab or locally.
    - Installs missing packages only (unless force_reinstall=True).
    - Ensures Ray Tune dependencies are installed even if ray is already present.
    - Can pin Ray version with ray_version (e.g., "1.12.0").
    """

    # Base list of packages
    packages = [
        "torch", "torchvision", "torchaudio",
        "ray[tune]", "tensorboardX", "hyperopt",
        "numpy", "pandas", "matplotlib",
        "scikit-image", "scipy"
    ]

    # Optional packages for visualization
    optional_packages = ["tensorboard"]
    widgets_packages = ["ipywidgets"]
    missing = []

    for pkg in packages:
        pkg_name = pkg.split("[")[0]

        # Special handling for Ray Tune
        if pkg_name == "ray":
            try:
                import ray
                import ray.tune
                ray_tune_installed = True
            except ImportError:
                ray_tune_installed = False

            # Build the package name with version if specified
            if ray_version:
                pkg = f"ray[{ 'tune' if 'tune' in pkg else ''}]=={ray_version}"

            if force_reinstall or not ray_tune_installed:
                missing.append(pkg)
            continue

        # General case
        if importlib.util.find_spec(pkg_name) is None or force_reinstall:
            missing.append(pkg)

    # Optionally add optional packages
    if include_optional:
        missing += optional_packages + widgets_packages

    if not missing:
        print("✅ All required packages already installed.")
        return

    print(f"📦 Installing missing packages: {', '.join(missing)}")

    # Build pip command
    if IN_COLAB:
        cmd = ["pip", "install", "--upgrade"] + missing
    else:
        cmd = [sys.executable, "-m", "pip", "install", "--upgrade"] + missing

    try:
        subprocess.check_call(cmd)
        print("✅ Installation complete.")
    except subprocess.CalledProcessError as e:
        print(f"❌ Installation failed: {e}")

def install_required_packages_forceGPU(IN_COLAB=True, force_reinstall=False, include_optional=True):
    """
    Installs required Python packages efficiently.
    - Detects if running in Colab or locally.
    - Installs missing packages only (unless force_reinstall=True).
    - Automatically installs GPU-enabled PyTorch if a CUDA-capable GPU is detected.
    """

    # Base list of non-PyTorch packages
    other_packages = [
        "ray[tune]", "tensorboardX", "hyperopt",
        "numpy", "pandas", "matplotlib",
        "scikit-image", "scipy"
    ]

    # Optional packages
    optional_packages = ["tensorboard"]
    widgets_packages = ["ipywidgets"]

    missing = []

    # ------------------------------
    # PyTorch: detect CPU vs GPU
    # ------------------------------
    torch_packages = ["torch", "torchvision", "torchaudio"]
    pip_index_url = []

    cuda_available = False
    try:
        import torch
        cuda_available = torch.cuda.is_available()
    except ImportError:
        pass

    if not IN_COLAB and not cuda_available:
        # Check GPU presence on Windows via nvidia-smi
        try:
            gpu_info = subprocess.check_output("nvidia-smi", shell=True).decode()
            if "failed" not in gpu_info.lower():
                cuda_available = True
        except Exception:
            pass

    # If local GPU detected, uninstall CPU-only PyTorch first and set CUDA URL
    if cuda_available and not IN_COLAB:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "torch", "torchvision", "torchaudio"])
        except subprocess.CalledProcessError:
            # Ignore if PyTorch is not installed
            pass
        pip_index_url = ["--index-url", "https://download.pytorch.org/whl/cu124"]

    # ------------------------------
    # Check other packages
    # ------------------------------
    for pkg in other_packages:
        pkg_name = pkg.split("[")[0]
        if pkg_name == "ray":
            try:
                import ray
                import ray.tune
                ray_tune_installed = True
            except ImportError:
                ray_tune_installed = False
            if force_reinstall or not ray_tune_installed:
                missing.append(pkg)
            continue

        if importlib.util.find_spec(pkg_name) is None or force_reinstall:
            missing.append(pkg)

    # Add optional packages
    if include_optional:
        missing += optional_packages + widgets_packages

    # Remove duplicates
    missing = list(dict.fromkeys(missing))

    if not missing and not force_reinstall:
        print("✅ All required packages already installed.")
        return

    print(f"📦 Installing missing packages: {', '.join(torch_packages + missing)}")

    # ------------------------------
    # Build pip command
    # ------------------------------
    if IN_COLAB:
        cmd = ["pip", "install", "--upgrade"] + torch_packages + missing + pip_index_url
    else:
        cmd = [sys.executable, "-m", "pip", "install", "--upgrade"] + torch_packages + missing + pip_index_url

    # ------------------------------
    # Run pip install
    # ------------------------------
    try:
        subprocess.check_call(cmd)
        print("✅ Installation complete.")
    except subprocess.CalledProcessError as e:
        print(f"❌ Installation failed: {e}")

    # ------------------------------
    # Confirm CUDA availability
    # ------------------------------
    try:
        import torch
        if torch.cuda.is_available():
            print(f"✅ PyTorch CUDA detected: {torch.cuda.get_device_name(0)}")
        else:
            print("⚠️ PyTorch installed, but no GPU detected. Using CPU-only build.")
    except ImportError:
        print("❌ PyTorch installation failed completely.")

def install_required_packages(IN_COLAB=True, force_reinstall=False, include_optional=True):
    """
    Installs required Python packages efficiently.
    - Detects if running in Colab or locally.
    - Installs missing packages only (unless force_reinstall=True).
    - Ensures Ray Tune dependencies are installed even if ray is already present.
    """

    # Base list of packages
    packages = [
        "torch", "torchvision", "torchaudio",
        "ray[tune]", "tensorboardX", "hyperopt",
        "numpy", "pandas", "matplotlib",
        "scikit-image", "scipy"
    ]

    # Optional packages for visualization
    optional_packages = ["tensorboard"]
    # Widgets for tqdm are optional; plain progress bar is fine
    widgets_packages = ["ipywidgets"]
    missing = []

    for pkg in packages:
        pkg_name = pkg.split("[")[0]

        # Special handling for Ray Tune
        if pkg_name == "ray":
            try:
                import ray
                import ray.tune
                ray_tune_installed = True
            except ImportError:
                ray_tune_installed = False
            if force_reinstall or not ray_tune_installed:
                missing.append(pkg)
            continue

        # General case
        if importlib.util.find_spec(pkg_name) is None or force_reinstall:
            missing.append(pkg)

    # Optionally add optional packages
    if include_optional:
        missing += optional_packages + widgets_packages

    if not missing:
        print("✅ All required packages already installed.")
        return

    print(f"📦 Installing missing packages: {', '.join(missing)}")

    # Build pip command
    if IN_COLAB:
        cmd = ["pip", "install", "--upgrade"] + missing
    else:
        cmd = [sys.executable, "-m", "pip", "install", "--upgrade"] + missing

    try:
        subprocess.check_call(cmd)
        print("✅ Installation complete.")
    except subprocess.CalledProcessError as e:
        print(f"❌ Installation failed: {e}")

def refresh_repo(
    IN_COLAB = True,
    repo_name: str = "FlexCNN_for_Medical_Physics",
    github_username: str = "petercl8",
    local_repo_path: str = None,
    auto_import: bool = True,
    verbose: bool = True):
    """
    Clone/pull and install the repo, then optionally auto-import all modules.
    Also reloads all submodules to reflect changes without restarting the runtime.
    """
    # --- Determine base directory ---
    base_dir = "/content" if IN_COLAB else local_repo_path
    if base_dir is None:
        raise ValueError("local_repo_path must be provided if not in Colab")

    repo_path = os.path.join(base_dir, repo_name)
    repo_url = (
        f"https://github.com/{github_username}/{repo_name}.git"
        if IN_COLAB
        else f"git@github.com:{github_username}/{repo_name}.git"
    )

    # --- Clone or update ---
    if not os.path.exists(repo_path):
        if verbose:
            print(f"📦 Cloning {repo_name} into {base_dir}...")
        subprocess.run(["git", "clone", repo_url], cwd=base_dir, check=True)
    else:
        if verbose:
            print(f"🔄 Pulling latest changes in {repo_path}...")
        subprocess.run(["git", "pull"], cwd=repo_path, check=True)

    # --- Install package in editable mode ---
    if verbose:
        print("⚙️ Installing the package in editable mode...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-e", "."],
                   cwd=repo_path, check=True)

    # --- Ensure repo path is importable ---
    if repo_path not in sys.path:
        sys.path.insert(0, repo_path)

    # --- Import the package ---
    package = importlib.import_module(repo_name)

    # --- Reload all submodules recursively ---
    def reload_submodules(pkg):
        for _, modname, ispkg in pkgutil.walk_packages(pkg.__path__, pkg.__name__ + "."):
            if modname in sys.modules:
                importlib.reload(sys.modules[modname])
            else:
                importlib.import_module(modname)
        importlib.reload(pkg)

    reload_submodules(package)

    # --- Gather all symbols ---
    imported = {}
    for _, modname, ispkg in pkgutil.walk_packages(package.__path__, package.__name__ + "."):
        mod = importlib.import_module(modname)
        for name, obj in inspect.getmembers(mod):
            if not name.startswith("_"):
                imported[name] = obj

    # --- Inject symbols into caller's globals if requested ---
    if auto_import:
        if verbose:
            print("✨ Injecting all symbols into global namespace...")
        caller_globals = inspect.stack()[1].frame.f_globals
        caller_globals.update(imported)
        if verbose:
            print(f"✅ Setup complete: {len(imported)} symbols loaded into globals.")
    else:
        if verbose:
            print(f"✅ Imported {len(imported)} symbols (not injected).")


In [7]:
# --- Sense environment ---
IN_COLAB = sense_colab()

# --- Install packages and import ---
install_required_packages(IN_COLAB)

📦 Installing missing packages: ray[tune], tensorboardX, scikit-image, tensorboard, ipywidgets
✅ Installation complete.


In [9]:
# --- Refresh Repository ---
refresh_repo(IN_COLAB, local_repo_path=local_repo_dirPath)

# --- Test Resources ---
list_compute_resources()

# --- Set main project directory
project_dirPath = setup_project_dirs(IN_COLAB, project_local_dirPath, project_colab_dirPath, mount_colab_drive=True)

# --- Set Device ---
device = sense_device(device=device_opt)

# Build grouped parameter dictionaries #

common_settings = {
    'run_mode': run_mode,
    'device': device,
    'num_examples': num_examples,
}

base_dirs = {
    'project_dirPath': project_dirPath,
    'plot_dirName': plot_dirName,
    'checkpoint_dirName': checkpoint_dirName,
    'tune_storage_dirName': tune_storage_dirName,
    'tune_dataframe_dirName': tune_dataframe_dirName,
    'test_dataframe_dirName': test_dataframe_dirName,
    'data_dirName': data_dirName
}

data_files = {
    'tune_sino_file': tune_sino_file,
    'tune_image_file': tune_image_file,
    'train_sino_file': train_sino_file,
    'train_image_file': train_image_file,
    'test_sino_file': test_sino_file,
    'test_image_file': test_image_file
}

mode_files = {
    'tune_csv_file': tune_csv_file,
    'train_checkpoint_file': train_checkpoint_file,
    'test_checkpoint_file': test_checkpoint_file,
    'test_csv_file': test_csv_file,
    'visualize_checkpoint_file': visualize_checkpoint_file
}

network_opts = {
    'network_type': network_type,
    'train_SI': train_SI,
    'image_size': image_size,
    'sino_size': sino_size,
    'image_channels': image_channels,
    'sino_channels': sino_channels,
}

tune_opts = {
    'tune_exp_name': tune_exp_name,
    'tune_scheduler': tune_scheduler,
    'tune_dataframe_fraction': tune_dataframe_fraction,
    'tune_restore': tune_restore,
    'tune_max_t': tune_max_t,
    'tune_minutes': tune_minutes,
    'tune_for': tune_for,
    'tune_even_reporting': tune_even_reporting,
    'tune_batches_per_report': tune_batches_per_report,
    'tune_examples_per_report': tune_examples_per_report,
    'tune_augment': tune_augment,
    'tune_grace_period': tune_grace_period,
    'tune_debug': tune_debug,
    'tune_force_fixed_config': tune_force_fixed_config,
    'num_CPUs': num_CPUs,
    'num_GPUs': num_GPUs,
    'cpus_per_trial': CPUs_per_trial,  # per trial
    'gpus_per_trial': GPUs_per_trial,  # per trial
}

train_opts = {
    'train_load_state': train_load_state,
    'train_save_state': train_save_state,
    'training_epochs': train_epochs,
    'train_augment': train_augment,
    'train_display_step': train_display_step,
    'train_sample_division': train_sample_division,
    'train_show_times': train_show_times,
}

test_opts = {
    'test_display_step': test_display_step,
    'test_batch_size': test_batch_size,
    'test_chunk_size': test_chunk_size,
    'testset_size': testset_size,
    'test_begin_at': test_begin_at,
    'test_compute_MLEM': test_compute_MLEM,
    'test_set_type': test_set_type,
    'test_merge_dataframes': test_merge_dataframes,
    'test_show_times': test_show_times,
    'test_shuffle': test_sample_division,
    'test_sample_division': test_sample_division
}

viz_opts = {
    'visualize_batch_size': visualize_batch_size,
    'visualize_offset': visualize_offset,
    'visualize_type': visualize_type,
    'visualize_shuffle': visualize_shuffle,
}

# Build paths and settings using new functions
paths = setup_paths(
    run_mode=run_mode,
    base_dirs=base_dirs,
    data_files=data_files,
    mode_files=mode_files,
    test_ops=test_opts,
    viz_ops=viz_opts
)

settings = setup_settings(
    run_mode=run_mode,
    common_settings=common_settings,
    tune_opts=tune_opts,
    train_opts=train_opts,
    test_opts=test_opts,
    viz_opts=viz_opts,
)

# --- Build Config Dictionary ---
config = construct_config(
    run_mode=run_mode,
    network_opts=network_opts,
    test_opts=test_opts,
    viz_opts=viz_opts,
    config_SUP_SI=config_SUP_SI,
    config_SUP_IS=config_SUP_IS,
    config_GAN_SI=config_GAN_SI,
    config_GAN_IS=config_GAN_IS,
    config_CYCLEGAN=config_CYCLEGAN,
    config_CYCLESUP=config_CYCLESUP,
    config_RAY_SI=config_RAY_SI,
    config_RAY_IS=config_RAY_IS,
    config_RAY_SUP=config_RAY_SUP,
    config_RAY_GAN=config_RAY_GAN,
    config_SUP_RAY_cycle=config_SUP_RAY_cycle,
    config_GAN_RAY_cycle=config_GAN_RAY_cycle,
)

tune_exp()

🔄 Pulling latest changes in /content/FlexCNN_for_Medical_Physics...
⚙️ Installing the package in editable mode...
✨ Injecting all symbols into global namespace...
✅ Setup complete: 135 symbols loaded into globals.
CPUs available: 12
GPUs available: 1
  GPU 0: NVIDIA L4
Mounted at /content/drive
placeholder


In [10]:

from ray import air, tune, train
from ray.air import session
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import FIFOScheduler # First in/first out scheduler
from ray.tune import ResultGrid, JupyterNotebookReporter, CLIReporter
from ray.tune.search.hyperopt import HyperOptSearch    # Search Algorithm (current)
#from ray.tune.suggest.ax import AxSearch               # Search Algorithm (couldn't make this work)
#from ray.tune.suggest.bayesopt import BayesOptSearch   # Search Algorithm (couldn't make this work)

# Pytorch ##
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
torch.manual_seed(0)  # For testing purposes

## Torchvision ##
from torchvision.utils import make_grid
from torchvision import transforms

## Numpy/MatPlotLib ##
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.pyplot import savefig
import matplotlib.gridspec as gridspec

## Pandas ##
import pandas as pd

## SciKit #
from skimage import metrics
from skimage.metrics import structural_similarity
from skimage.transform import radon, iradon
from skimage.transform import iradon
from skimage import morphology
from skimage.morphology import opening, erosion
#from skimage.restoration import denoise_bilateral, denoise_tv_chambolle, denoise_wavelet

## SciPy ##
#from scipy.stats import moment as compute_moment

## Python ##
import os
import time
import sys

#from IPython.display import display, clear_output


In [11]:
print('Pre-tune check:')
print('  num_CPUs:', tune_opts.get('num_CPUs'))
print('  num_GPUs:', tune_opts.get('num_GPUs'))
print('  cpus_per_trial:', tune_opts.get('cpus_per_trial'))
print('  gpus_per_trial:', tune_opts.get('gpus_per_trial'))
print('  settings device:', settings['device'])
import torch
print('  torch.cuda.is_available():', torch.cuda.is_available())
if torch.cuda.is_available():
    print('  CUDA device:', torch.cuda.get_device_name(0))

Pre-tune check:
  num_CPUs: 12
  num_GPUs: 1
  cpus_per_trial: 2
  gpus_per_trial: 1
  settings device: cuda
  torch.cuda.is_available(): True
  CUDA device: NVIDIA L4


# Analysis

In [ ]:
# --- Refresh Repository ---
#refresh_repo(IN_COLAB, local_repo_path=local_repo_dirPath)

## Anthropomorphic phantoms ##
##############################
fig_size_XCAT=2.5
indexes_XCAT = [300, 305, 310, 315, 320, 325]

test_array_names = ['test-actMap.npy', 'test-highCountImage.npy', 'test-obliqueImage.npy']
#test_array_names = ['test-actMap.npy']
test_sino = 'test-highCountSino-382x513.npy'

train_array_names = ['train-actMap.npy', 'train-highCountImage.npy', 'train-obliqueImage.npy']
train_sino = 'train-highCountSino-382x513.npy'


## QA phantoms ##
#################
fig_size_QA=2.5
indexes_QA = [12, 13, 14, 15, 16, 17, 18, 19, 20]

NEMA_array_names = ['QA-NEMA-actMap.npy', 'QA-NEMA-highCountImage.npy', 'QA-NEMA-obliqueImage.npy']
NEMA_sino = 'QA-NEMA-highCountSino.npy'

Pinwheel_array_names = ['QA-Pinwheel-actMap.npy', 'QA-Pinwheel-highCountImage.npy', 'QA-Pinwheel-obliqueImage.npy']
Pinwheel_sino = 'QA-Pinwheel-highCountSino.npy'


#actMap = 'QA-Radial-actMap.npy'
#sino = 'QA-Radial-highCountSino.npy'

#actMap = 'QA-Axial-actMap.npy'
#sino = 'QA-Axial-highCountSino.npy'

## Network Configs ##
#####################

checkpoint_fileName_90x90 = 'checkpoint-new_data-old_net-90x90-100epochs'
config_90x90 = {
  "SI_dropout": False,  "SI_exp_kernel": 4,  "SI_gen_fill": 0,  "SI_gen_final_activ": nn.Tanh(),  "SI_gen_hidden_dim": 23,
  "SI_gen_mult": 1.6605902406330195,  "SI_gen_neck": 5,  "SI_gen_z_dim": 789,  "SI_layer_norm": "instance",  "SI_normalize": True,  "SI_pad_mode": "zeros",  "SI_scale_fixed": 8100,  "batch_size": 71,  "gen_b1": 0.2082092731474774,  "gen_b2": 0.27147903136187507,
  "gen_lr": 0.0005481469822215635, "sup_criterion": nn.MSELoss(),
  "sino_size":90, "sino_channels": 3, "image_channels":1, "image_size":90, "train_SI": True, "network_type": "SUP"
}

checkpoint_fileName_180x180 = 'checkpoint-new_data-old_net-180x180-100epochs'
config_180x180 = {
  "SI_dropout": False,  "SI_exp_kernel": 4,  "SI_gen_fill": 0,  "SI_gen_final_activ": nn.Tanh(),  "SI_gen_hidden_dim": 23,
  "SI_gen_mult": 1.6605902406330195,  "SI_gen_neck": 5,  "SI_gen_z_dim": 789,  "SI_layer_norm": "instance",  "SI_normalize": True,  "SI_pad_mode": "zeros",  "SI_scale_fixed": 8100,  "batch_size": 71,  "gen_b1": 0.2082092731474774,  "gen_b2": 0.27147903136187507,
  "gen_lr": 0.0005481469822215635, "sup_criterion": nn.MSELoss(),
  "sino_size":180, "sino_channels": 3, "image_channels":1, "image_size":90, "train_SI": True, "network_type": "SUP"
}


### Plots Images ###
####################
image_tensor, sino_tensor = PlotPhantomRecons(test_array_names, test_sino, config_90x90, paths, indexes_XCAT, checkpoint_fileName_90x90, fig_size_XCAT, device)
image_tensor, sino_tensor = PlotPhantomRecons(test_array_names, test_sino, config_180x180, paths, indexes_XCAT, checkpoint_fileName_180x180, fig_size_XCAT, device)

#image_tensor, sino_tensor = PlotPhantomRecons(NEMA_array_names, NEMA_sino, config_90x90, paths, indexes, checkpoint_fileName_90x90, fig_size_QA, device)
#image_tensor, sino_tensor = PlotPhantomRecons(Pinwheel_array_names, Pinwheel_sino, config_90x90, paths, indexes, checkpoint_fileName_90x90, fig_size_QA, device)


#############
## Metrics ##
#############

#frame_SSIM_MLEM, placeholder = calculate_metric(MLEM_output, image_tensor, SSIM, dataframe = True, label='MLEM, SSIM')
#frame_MSE_MLEM, placeholder =  calculate_metric(MLEM_output, image_tensor, MSE, dataframe = True, label='MLEM, MSE')
#print('################### MLEM ###################')
#print(frame_SSIM_MLEM.T)
#print(frame_MSE_MLEM.T)

In [ ]:
#################
## Set Options ##
#################

tune_exp_name='search-Quartile-lowSSIM-tunedSSIM-D'
save_fig=False

titlesize=13
fontsize=12
ticksize=10
dpi=800
figsize=(10,8)

fig = plt.figure(figsize=figsize, dpi=dpi)
gs = gridspec.GridSpec(ncols=100, nrows=100)

# Top Row Axes #
ax1 = fig.add_subplot(gs[0:25,   0:100])
ax2 = fig.add_subplot(gs[38:62,   0:100])
ax3 = fig.add_subplot(gs[75:100,  0:100])


## Plots ###

refresh_repo(IN_COLAB, local_repo_path=local_repo_dirPath)

result_grid, bestResult_logDir = PlotFrame(paths, tune_exp_name, ax1, 'batch_step', 'Batch Step', 'MSE', 'MSE', ylim=(4,20), logy=True)
ax1.set_title('(A) MSE Learning Curves', fontsize=titlesize)

result_grid, bestResult_logDir = PlotFrame(paths, tune_exp_name, ax2, 'batch_step', 'Batch Step', 'SSIM', 'SSIM', ylim=(0,0.8), logy=False)
ax2.set_title('(B) SSIM Learning Curves', fontsize=titlesize)

result_grid, bestResult_logDir = PlotFrame(paths, tune_exp_name, ax3, 'batch_step', 'Batch Step', 'CUSTOM', 'Local Distributions Metric', ylim=(300,500))
ax3.set_title('(A) LDM Learning Curves', fontsize=titlesize)

# Best Result #

print(bestResult_logdir)

# Save Fig? #

if save_fig:
    plot_save_name='figure-tuning'
    savefig(os.path.join(paths['plot_dirPath'], plot_save_name+'.svg'), bbox_inches='tight')

In [ ]:
print(paths)

## Load: Test Dataframes


### Plot Test Dataframes

# Sort: Dataset by Metric

### Save Datasets & Check

In [ ]:
#save_sino_path = '/content/drive/MyDrive/Repository/PET_Data/quartile_data/train_sino-lowSSIM-17500.npy'
#save_image_path = '/content/drive/MyDrive/Repository/PET_Data/quartile_data/train_sino-lowSSIM-17500.npy'

# Print sorted array shape & display a few images #
print('save_sino_array.shape: ', save_sino_array.shape)
print('save_image_array.shape: ', save_image_array.shape)

print('save_sino_array sample images')
print('save_image_array sample images')
show_multiple_matched_tensors(torch.from_numpy(save_sino_array[500:509]))
show_multiple_matched_tensors(torch.from_numpy(save_image_array[500:509]))


# Save the sorted array to disk #
save_sino_array.flush()
save_image_array.flush()
#np.save(save_sino_path, save_sino_array)
#np.save(save_image_path, save_image_array)

# Load the saved array and make sure it's the same size/has the same images #
load_sino_array = np.load(save_sino_path, mmap_mode='r')
load_image_array = np.load(save_image_path, mmap_mode='r')
print('load_sino_array.shape: ', load_sino_array.shape)
print('load_image_array.shape: ', load_image_array.shape)

print('load_sino_array sample images')
print('load_image_array sample images')
show_multiple_matched_tensors(torch.from_numpy(load_sino_array[500:509]))
show_multiple_matched_tensors(torch.from_numpy(load_image_array[500:509]))


In [ ]:
## Find what GPU I'm using ##
import torch
print(torch.cuda.is_available())  # Should be True

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tensor board works for all experiments except the last one.
My plotting function no longer works for any of the experiments.